<b>Getting Started</b>

- Using a library called MoviePy, we will extract the audio from the video recording. And in the next step, we will convert that audio file into text using Google’s speech recognition library. If you are ready, let’s get started by installing the libraries!


<img src='https://edusync.s3.ams03.cloud-object-storage.appdomain.cloud/Video-2-Text.jpg' height='200'>

### Libraries<br/>

   - Speech Recognition
   - MoviePy

In [ ]:
pip install SpeechRecognition moviepy


<b>SpeechRecognition module supports multiple recognition APIs, and Google Speech API is one of them</b>

<b>MoviePy is a library that can read and write all the most common audio and video formats, including GIF. </b>
- If you are having issues when installing moviepy library, try by installing ffmpeg. Ffmpeg is a leading multimedia framework, able to decode, encode, transcode, mux, demux, stream, filter and play pretty much anything that humans and machines have created.

### Step 1 — Import Libraries

In [ ]:
import speech_recognition as sr 
import moviepy.editor as mp

### Step 2 — Video to Audio Conversion

In [ ]:
clip = mp.VideoFileClip(r”video_recording.mov”) 
 
clip.audio.write_audiofile(r”converted.wav”)


### Step 3 — Speech Recognition

In [ ]:
r = sr.Recognizer()
audio = sr.AudioFile("converted.wav")


The best part, which is recognizing the speech in an audio file. The recognizer will try to understand the speech and convert it to a text format.

In [ ]:
with audio as source:
  audio_file = r.record(source)
result = r.recognize_google(audio_file)

### Exporting the intermediate Result


In [ ]:
# exporting the result 
with open('recognized.txt',mode ='w') as file: 
   file.write("Recognized Speech:") 
   file.write("\n") 
   file.write(result) 
   print("ready!")

### Convert Experience  from speech to text

In [ ]:
# coding=utf-8
import json
from os.path import join, dirname
from ibm_watson import TextToSpeechV1
from ibm_watson.websocket import SynthesizeCallback
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('your_api_key')
service = TextToSpeechV1(authenticator=authenticator)
service.set_service_url('https://api.us-south.text-to-speech.watson.cloud.ibm.com')

voices = service.list_voices().get_result()
print(json.dumps(voices, indent=2))

with open(join(dirname(__file__), '../resources/output.wav'),
          'wb') as audio_file:
    response = service.synthesize(
        'Hello world!', accept='audio/wav',
        voice="en-US_AllisonVoice").get_result()
    audio_file.write(response.content)

pronunciation = service.get_pronunciation('Watson', format='spr').get_result()
print(json.dumps(pronunciation, indent=2))

voice_models = service.list_custom_models().get_result()
print(json.dumps(voice_models, indent=2))

# voice_model = service.create_custom_model('test-customization').get_result()
# print(json.dumps(custom_model, indent=2))

# updated_custom_model = service.update_custom_model(
#     'YOUR CUSTOMIZATION ID', name='new name').get_result()
# print(updated_custom_model)


file_path = join(dirname(__file__), "../resources/dog.wav")
class MySynthesizeCallback(SynthesizeCallback):
    def __init__(self):
        SynthesizeCallback.__init__(self)
        self.fd = open(file_path, 'ab')

    def on_connected(self):
        print('Connection was successful')

    def on_error(self, error):
        print('Error received: {}'.format(error))

    def on_content_type(self, content_type):
        print('Content type: {}'.format(content_type))

    def on_timing_information(self, timing_information):
        print(timing_information)

    def on_audio_stream(self, audio_stream):
        self.fd.write(audio_stream)

    def on_close(self):
        self.fd.close()
        print('Done synthesizing. Closing the connection')

my_callback = MySynthesizeCallback()
service.synthesize_using_websocket('I like to pet dogs',
                                   my_callback,
                                   accept='audio/wav',
                                   voice='en-US_AllisonVoice'
                                  )

### Experiences sharing on EduSync Watson Assistant

In [ ]:
import json
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('your apikey')
assistant = AssistantV2(
    version='2018-09-20',
    authenticator=authenticator)
assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com')

#########################
# Sessions
#########################

session = assistant.create_session("<YOUR ASSISTANT ID>").get_result()
print(json.dumps(session, indent=2))

assistant.delete_session("<YOUR ASSISTANT ID>", "<YOUR SESSION ID>").get_result()

#########################
# Message
#########################

message = assistant.message(
    "<YOUR ASSISTANT ID>",
    "<YOUR SESSION ID>",
    input={'text': 'What\'s the weather like?'},
    context={
        'metadata': {
            'deployment': 'myDeployment'
        }
    }).get_result()
print(json.dumps(message, indent=2))

# logs = assistant.list_logs(
#     "<YOUR ASSISTANT ID>"
# )
# print(json.dumps(logs, indent=2))

### Experiences Language Processing

In [ ]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions
# from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# Authentication via IAM
# authenticator = IAMAuthenticator('your_api_key')
# service = NaturalLanguageUnderstandingV1(
#     version='2018-03-16',
#     authenticator=authenticator)
# service.set_service_url('https://gateway.watsonplatform.net/natural-language-understanding/api')

# Authentication via external config like VCAP_SERVICES
service = NaturalLanguageUnderstandingV1(
    version='2018-03-16')
service.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com')

response = service.analyze(
    text='Bruce Banner is the Hulk and Bruce Wayne is BATMAN! '
    'Superman fears not Banner, but Wayne.',
    features=Features(entities=EntitiesOptions(),
                      keywords=KeywordsOptions())).get_result()

print(json.dumps(response, indent=2))

###   Visual Recognition on Experience sharing

In [ ]:
import json
import os
from ibm_watson import VisualRecognitionV4
from ibm_watson.visual_recognition_v4 import FileWithMetadata, TrainingDataObject, Location, AnalyzeEnums
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(
    'YOUR APIKEY')
service = VisualRecognitionV4(
    '2018-03-19',
    authenticator=authenticator)
service.set_service_url('https://api.us-south.visual-recognition.watson.cloud.ibm.com')

# create a classifier
my_collection = service.create_collection(
    name='',
    description='testing for python'
).get_result()
collection_id = my_collection.get('collection_id')

# add images
with open(os.path.join(os.path.dirname(__file__), '../resources/South_Africa_Luca_Galuzzi_2004.jpeg'), 'rb') as giraffe_info:
    add_images_result = service.add_images(
        collection_id,
        images_file=[FileWithMetadata(giraffe_info)],
    ).get_result()
print(json.dumps(add_images_result, indent=2))
image_id = add_images_result.get('images')[0].get('image_id')

# add image training data
training_data = service.add_image_training_data(
    collection_id,
    image_id,
    objects=[
        TrainingDataObject(object='giraffe training data',
                           location=Location(64, 270, 755, 784))
    ]).get_result()
print(json.dumps(training_data, indent=2))

# update object metadata
updated_object_metadata = service.update_object_metadata(
    collection_id=collection_id,
    object='giraffe training data',
    new_object='updated giraffe training data').get_result()
print(json.dumps(updated_object_metadata, indent=2))

# train collection
train_result = service.train(collection_id).get_result()
print(json.dumps(train_result, indent=2))

# training usage
training_usage = service.get_training_usage()
print(json.dumps(training_usage, indent=2))

# analyze
dog_path = os.path.join(os.path.dirname(__file__), '../resources/dog.jpg')
giraffe_path = os.path.join(os.path.dirname(__file__), '../resources/my-giraffe.jpeg')
with open(dog_path, 'rb') as dog_file, open(giraffe_path, 'rb') as giraffe_files:
    analyze_images = service.analyze(
        collection_ids=[collection_id],
        features=[AnalyzeEnums.Features.OBJECTS.value],
        images_file=[
            FileWithMetadata(dog_file),
            FileWithMetadata(giraffe_files)
        ],
        image_url=['https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/American_Eskimo_Dog.jpg/1280px-American_Eskimo_Dog.jpg']).get_result()
    print(json.dumps(analyze_images, indent=2))

# delete collection
service.delete_collection(collection_id)